In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
import copy
import json
import time
import os
from os.path import join
import pickle as pkl

import sys
sys.path.append('../../')

from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.sensitivity import calculate_sensitivity_and_plot
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, scale_up_testing_and_forecast
from main.seir.forecast import create_all_csvs, create_decile_csv, create_region_csv, write_csv
from main.seir.uncertainty import MCUncertainty
from utils.create_report import create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from utils.enums.columns import *

In [ ]:
state, dist = ('Maharashtra', 'Mumbai')

In [ ]:
output_folder = '../../reports/{}'.format("1594480618.481472")

In [ ]:
with open(join(output_folder, 'predictions_dict.pkl'), 'rb') as f:
    predictions_dict = pkl.load(f)

In [ ]:
predictions_dict['m2']['forecasts'].keys()

In [ ]:
df_true = get_data(state=state, district=dist, disable_tracker=False, use_dataframe='data_all', filename=None)

In [ ]:
df_true

In [ ]:
from viz import axis_formatter

In [ ]:
predictions_dict['m2']['df_district']

In [ ]:
predictions_dict['m2']['forecasts'][80]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

df_true_plotting = df_true
df_predicted_plotting = predictions_dict['m2']['forecasts'][80]
df_train = predictions_dict['m2']['df_train']
df_val = predictions_dict['m2']['df_val']

train_period = predictions_dict['m2']['run_params']['train_period']

for compartment in compartments['base']:
    ax.plot(df_true_plotting[compartments['date'][0].name].to_numpy(), 
            df_true_plotting[compartment.name].to_numpy(),
            '-o', color=compartment.color, label='{} (Observed)'.format(compartment.label))
    ax.plot(df_predicted_plotting[compartments['date'][0].name].to_numpy(), 
            df_predicted_plotting[compartment.name].to_numpy(),
            '-.', color=compartment.color, label='{} (Predicted)'.format(compartment.label))

ax.axvline(x=df_train.iloc[-train_period, :]['date'], ls=':', color='brown', label='Train starts')
ax.axvline(x=df_train.iloc[-1, :]['date'], ls=':', color='black', label='Last data point seen by model')

axis_formatter(ax, None, custom_legend=False)
fig.suptitle('Predictions of 80th Percentile vs Ground Truth (Unseen Data)')
plt.tight_layout()

In [ ]:
df_true_plotting = df_true
df_predicted_plotting = predictions_dict['m2']['forecasts'][80]
df_train = predictions_dict['m2']['df_train']
df_val = predictions_dict['m2']['df_val']

train_period = predictions_dict['m2']['run_params']['train_period']

fig, axs = plt.subplots(figsize=(18, 12), nrows=2, ncols=2)
for i, ax in enumerate(axs.flat):
    compartment = compartments['base'][i]
    ax.plot(df_true_plotting[compartments['date'][0].name].to_numpy(), 
            df_true_plotting[compartment.name].to_numpy(),
            '-o', color=compartment.color, label='{} (Observed)'.format(compartment.label))
    ax.plot(df_predicted_plotting[compartments['date'][0].name].to_numpy(), 
            df_predicted_plotting[compartment.name].to_numpy(),
            '-.', color=compartment.color, label='{} (Predicted)'.format(compartment.label))

    ax.axvline(x=df_train.iloc[-train_period, :]['date'], ls=':', color='brown', label='Train starts')
    ax.axvline(x=df_train.iloc[-1, :]['date'], ls=':', color='black', label='Last data point seen by model')

    axis_formatter(ax, None, custom_legend=False)

fig.suptitle('Predictions of 80th Percentile vs Ground Truth')
# plt.tight_layout()

In [ ]:
df_train

In [ ]:
lc = Loss_Calculator()
forecast_errors_dict = {}

for key, df_prediction in predictions_dict['m2']['forecasts'].items():
    if key in ['best', 50, 80]:
        df_prediction = df_prediction.loc[(df_prediction['date'] > df_train.iloc[-1, :]['date']) & 
                                          (df_prediction['date'] <= df_true.iloc[-1, :]['date'])]
        df_true = df_true.loc[df_true['date'] > df_train.iloc[-1, :]['date']]
        df_prediction.reset_index(inplace=True, drop=True)
        df_true.reset_index(inplace=True, drop=True)
        ld = lc.calc_loss_dict(df_prediction, df_true, method='mape')
        ld = {key : round(value, 2) for key, value in ld.items()}
        forecast_errors_dict[key] = ld

In [ ]:
pd.DataFrame.from_dict(forecast_errors_dict)